БД: `PostgreSQL` \
Описание таблиц БД `db_test`:

1. Таблица `TRANSACTION_TIN` - содержит информацию о транзакциях клиентов. Столбцы:
- `transaction_id` (INT): Уникальный идентификатор транзакции.
- `customer_id` (INT): Уникальный идентификатор клиента (связь с таблицей CUSTOMER_TIN).
- `amount_rur` (DECIMAL(18, 2)): Сумма транзакции в российских рублях.
- `success_flg` (BOOLEAN): Флаг успешности транзакции (true - успешно, false - неуспешно).
- `transaction_dttm` (TIMESTAMP): Дата и время транзакции.

2. Таблица `CUSTOMER_TIN` - cодержит информацию о клиентах. Столбцы:
- `customer_id` (INT): Уникальный идентификатор клиента.
- `name` (VARCHAR(255)): фамилия клиента.
- `start_dttm` (TIMESTAMP): Дата и время начала действия записи о клиенте.
- `birth_dt` (DATE): Дата рождения клиента.

3. Таблица `CLIENT_TIN` - содержит информацию о клиентах. Столбцы:
- `id` (INT): Уникальный идентификатор клиента.
- `name` (VARCHAR(255)): фамилия клиента.

4. Таблица `ACCOUNT_TIN` - содержит информацию о счетах клиентов. Столбцы:
- `id` (INT): Уникальный идентификатор счета.
- `client_id` (INT): Уникальный идентификатор клиента (связь с таблицей CLIENT_TIN).
- `open_dt` (DATE): Дата открытия счета.
- `close_dt` (DATE): Дата закрытия счета (если счет закрыт).

5. Таблица `TRANSACTION_TIN2` - содержит информацию о дополнительных транзакциях клиентов. Пригодится в последней задаче. Столбцы:
- `id` (INT): Уникальный идентификатор транзакции.
- `account_id` (INT): Уникальный идентификатор счета.
- `transaction_date` (DATE): Дата транзакции.
- `amount` (DECIMAL(10, 2)): Сумма транзакции.
- `type_t` (INT): Тип транзакции (1 - начисления, 2 - списания).

### Задача 1
Вывести клиентов, сумма успешных транзакций которых более 100 тыс. 

Столбцы в результате: \
`customer_id`, \
`amount`

### Решение:
```
SELECT 
    customer_id, 
    amount
FROM 
    db_test.TRANSACTION_TIN
WHERE 
    success_flg IS TRUE
AND customer_id in (
        SELECT 
            customer_id
        FROM 
            db_test.TRANSACTION_TIN
        WHERE 
            success_flg IS TRUE
        GROUP BY 
            customer_id
        HAVING SUM
            (amount_rur) > 100000
)
ORDER BY 
    customer_id
```

### Задача 2
Вывести дату первой транзакции для клиентов, пришедших в 2023 году.

Столбцы в результате: \
`customer_id`, \
`transaction_dt `

### Решение:
```
WITH transaction_data as (
    SELECT 
        customer_id, 
        transaction_dttm as transaction_dt,
        ROW_NUMBER() OVER(partition by customer_id order by transaction_dttm::date) as rw
    FROM 
        db_test.TRANSACTION_TIN
    WHERE 
        success_flg is true
    order by 
        customer_id, rw
)
SELECT 
    customer_id, 
    transaction_dt
FROM 
    transaction_data
WHERE 
    rw = 1
AND customer_id in (
        SELECT 
            customer_id
        FROM 
            db_test.CUSTOMER_TIN
        WHERE 
            EXTRACT(year FROM start_dttm) = 2023)
```

### Задача 3
Вывести все транзакции, которые больше предыдущей и последующей транзакции по этому клиенту. \
Примечание: рассматриваем только успешные транзакции.

Столбцы в результате: \
`transaction_id`, \
`customer_id`

### Решение:
```
WITH transactions AS (
SELECT 
    customer_id,
    transaction_id,
    amount_rur,
    LAG(amount_rur) OVER (partition by customer_id order by transaction_id) AS prev_amount,
    LEAD(amount_rur) OVER (partition by customer_id order by transaction_id) AS next_amount
FROM 
    db_test.TRANSACTION_TIN
WHERE 
    success_flg is true
)
SELECT
    transaction_id, 
    customer_id
FROM 
    transactions
WHERE 
    amount_rur > prev_amount AND amount_rur > next_amount
```

### Задача 4
Для каждой транзакции посчитать количество дней, которые прошли с момента первой транзакции по этому клиенту. \
Проставить флаг, относится ли транзакция к клиенту, начавшему обслуживаться в банке в текущем году (2023).

Столбцы в результате: \
`transaction_id`, \
`customer_id`, \
`amount`, \
`transaction_dttm`, \
`days_count`, \
`new_cust_flg`

### Решение:
```
SELECT
  t.transaction_id, 
  t.customer_id, 
  t.amount_rur, 
  t.transaction_dttm, 
  EXTRACT(days FROM t.transaction_dttm - first_value(t.transaction_dttm) OVER(partition by t.customer_id order by t.transaction_dttm)
          ) AS days_count, 
  EXTRACT(year FROM c.start_dttm) = EXTRACT(year FROM NOW()) AS new_cust_flg 
FROM 
  db_test.TRANSACTION_TIN AS t 
INNER JOIN 
  interview.CUSTOMER_TIN AS c ON t.customer_id = c.customer_id 
WHERE 
  success_flg IS TRUE
```

### Задача 5
Вывести имена клиентов, у которых есть как минимум один активный счет, открытый больше года назад (ранее 1 июня 2022 года). И которые за последние полгода (с 1 июня 2022 года по 1 июня 2023 года) по всем своим счетам имеют движение средств меньше, чем 7000 рублей.

Примечание: движением средств считаем сумму всех транзакций клиента (как начислений, так и списаний, без учета знаков).

Столбцы в результате: \
`name`, \
`sum_amount `

### Решение:
```
SELECT
    c.name, 
    SUM(CASE WHEN t.type_t = 1 THEN t.amount ELSE 0 END) - SUM (CASE WHEN t.type_t = 2 THEN t.amount ELSE 0 END) AS sum_amount 
FROM 
    db_test.TRANSACTION_TIN2 AS t 
INNER JOIN 
    db_test.account_tin AS a ON a.id = t.account_id 
INNER JOIN 
    db_test.client_tin AS c ON c.id = a.client_id 
WHERE 
    transaction_date BETWEEN '2022-06-01'::timestamp
    AND '2023-06-01'::timestamp 
    AND a.client_id IN (
        SELECT 
            client_id 
        FROM 
            db_test.ACCOUNT_TIN 
        WHERE 
            close_dt IS NULL AND open_dt < '2022-01-06') 
GROUP BY 
    c.name 
HAVING 
    SUM(CASE WHEN t.type_t = 1 THEN t.amount ELSE 0 END) - SUM (CASE WHEN t.type_t = 2 THEN t.amount ELSE 0 END) < 7000
```